In [1]:
#include "TChain.h"

TChain* ch_1 = new TChain("tree");

// int start = 16;
int start = 25;
int stop = 25;

// for (int i= start; i<=stop; i++){
//   TString fileName = TString::Format("~/data/25e04/run%02d_00_hit.root" , i);
//   ch_1->AddFile(fileName.Data());
// }

ch_1->AddFile("~/data/25e04/10Be/hit/run_all_hit.root");
// ch_1->Print();

// std::cout << "OK\n";

In [2]:
Double_t Rxa[5];      // Raw Amplitude (Top 5)
Double_t Rxc[5];     // Calibrated energy（Top 5）
Int_t Rxa_n[5];        // Corresponding strip number (channel number)
Int_t Rxa_hits;           // Actual effective hit number

// Right y_strip silicon
Double_t Rya[5];    
Double_t Ryc[5];   
Int_t Rya_n[5];     
Int_t Rya_hits;        

// Left x_strip silicon
Double_t Lxa[5];    
Double_t Lxc[5];    
Int_t Lxa_n[5];     
Int_t Lxa_hits;        

// Left y_strip silicon
Double_t Lya[5];
Double_t Lyc[5];  
Int_t Lya_n[5];    
Int_t Lya_hits;       

// Right scintillator crystals
Double_t Rea[5];   
Int_t Rea_n[5];      
Int_t Rea_hits;

// Left scintillator crystals
Double_t Lea[5];
Int_t Lea_n[5]; 
Int_t Lea_hits;           

// TOF
Double_t ToF;
Double_t af3;
Double_t af5;

// Beam profile
Double_t xbt;
Double_t ybt;
Double_t xbd;
Double_t ybd;

ch_1->SetBranchAddress("Rxa", Rxa);
ch_1->SetBranchAddress("Rxc", Rxc);
ch_1->SetBranchAddress("Rxa_n", Rxa_n);
ch_1->SetBranchAddress("Rxa_hits", &Rxa_hits);

ch_1->SetBranchAddress("Rya", Rya);
ch_1->SetBranchAddress("Ryc", Ryc);
ch_1->SetBranchAddress("Rya_n", Rya_n);
ch_1->SetBranchAddress("Rya_hits", &Rya_hits);

ch_1->SetBranchAddress("Lxa", Lxa);
ch_1->SetBranchAddress("Lxc", Lxc);
ch_1->SetBranchAddress("Lxa_n", Lxa_n);
ch_1->SetBranchAddress("Lxa_hits", &Lxa_hits);

ch_1->SetBranchAddress("Lya", Lya);
ch_1->SetBranchAddress("Lyc", Lyc);
ch_1->SetBranchAddress("Lya_n", Lya_n);
ch_1->SetBranchAddress("Lya_hits", &Lya_hits);

ch_1->SetBranchAddress("Rea", Rea);
ch_1->SetBranchAddress("Rea_n", Rea_n);
ch_1->SetBranchAddress("Rea_hits", &Rea_hits);

ch_1->SetBranchAddress("Lea", Lea);
ch_1->SetBranchAddress("Lea_n", Lea_n);
ch_1->SetBranchAddress("Lea_hits", &Lea_hits);

ch_1->SetBranchAddress("ToF", &ToF);
ch_1->SetBranchAddress("af3", &af3);
ch_1->SetBranchAddress("af5", &af5);

In [3]:
// PID plots
TH2D* h_2d_1 = new TH2D("h_2d_1", "x strip", 1000, 0., 4000., 1000, 0., 50.);
TH2D* h_2d_2 = new TH2D("h_2d_2", "y strip", 1000, 0., 4000., 1000, 0., 50.);

// hit maps
TH2D* h_2d_3 = new TH2D("h_2d_3", "silicon coincidences", 16, 0., 16., 16, 0., 16);
TH2D* h_2d_4 = new TH2D("h_2d_4", "CsI(Tl) hits", 4, 0., 4., 4, 0., 4.);

TH2D* h_2d_5 = new TH2D("h_2d_3", "silicon coincidences", 16, 0., 16., 16, 0., 16);
TH2D* h_2d_6 = new TH2D("h_2d_4", "CsI(Tl) hits", 4, 0., 4., 4, 0., 4.);

Warning in <TROOT::Append>: Replacing existing TH1: h_2d_3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_2d_4 (Potential memory leak).


In [4]:
TCanvas* c_1 = new TCanvas("c_1","c_1");

/*c_1->cd();
ch_1->Draw
    (
        "NeEvent.Ryc[15]+NeEvent.Ryc[0]:NeEvent.Rea[0]>>h_1_c_1(1000,0,3500,1000,0,50)",
        "xbt*xbt+ybt*ybt<90",
        "colz"
    );
c_1->Draw();*/

int entries_used = ch_1->GetEntries();
// int entries_used = 1000000;
std::cout << "entries_used: " << entries_used << "\n";

int check_target = 0;
int check_ToF = 0;
int check_all;

double E_f_a;
double E_f_b;

// for (E_f_a = 0.5; E_f_a < 0.7; E_f_a)
// {
    for(int i = 0; i < entries_used; i++)
    {
        if(i%1000000==0){std::cout << "Entry: " << i << "\n";}
        ch_1->GetEntry(i);

        if(xbt*xbt+ybt*ybt<90.)
        {check_target = 1;}
        
        if(af3>1500. && af3<3000. && ToF>185. && ToF<200.)
        {check_ToF = 1;}

        check_all = check_target*check_ToF;

        if(check_all == 1 && Rxa_n[0] >= 0 && Rxa_n[0] <= 3 && Rya_n[0] >= 12 && Rxa_n[0] <= 15)
        // if(check_all == 1 && Lxa_n[0] >= 0 && Lxa_n[0] <= 10 && Lya_n[0] >= 0 && Lxa_n[0] <= 10)
        // if(check_all == 1 && Rxa_n[0] == 0 && Rya_n[0] == 15)
        {
            h_2d_1->Fill(Rea[0],Rxc[0]);    
            h_2d_2->Fill(Rea[0],Ryc[0]);  
            // h_2d_1->Fill(Lea[0],Lxc[0]);    
            // h_2d_2->Fill(Lea[0],Lyc[0]);        
            // h_2d_1->Fill(Rea[0],E_f);

            // double E_f = sqrt(Rxc[15]*Rea[0] + E_f_a*Rxc[15]*Rxc[15]) + E_f_b*Rea[0];    
        }

        if(Rxa_n[0]>=0 && Rya_n[0]>=0)
        {
            h_2d_3->Fill(Rxa_n[0], Rya_n[0]);
        }

        // Rea_n[0]=7;
        // std::cout << "No." << Rea_n[0]<< "\n";
        // std::cout << "x = " << Rea_n[0]%4 + 1 << "\n"; 
        // std::cout << "y = " << 4 - Rea_n[0]/4 << "\n";
        
        if(Rea_n[0]>=0)
        {
            h_2d_4->Fill(Rea_n[0]%4, 4 - Rea_n[0]/4 - 1);
        }
    }
// }
c_1->cd();
c_1->Divide(2,2);

c_1->cd(1);
h_2d_1->Draw("colz");

c_1->cd(2);
h_2d_2->Draw("colz");

c_1->cd(3);
h_2d_3->Draw("colz");

c_1->cd(4);
h_2d_4->Draw("colz");

c_1->Draw();

entries_used: 20000000
Entry: 0
Entry: 1000000
Entry: 2000000
Entry: 3000000
Entry: 4000000
Entry: 5000000
Entry: 6000000
Entry: 7000000
Entry: 8000000
Entry: 9000000
Entry: 10000000
Entry: 11000000
Entry: 12000000
Entry: 13000000
Entry: 14000000
Entry: 15000000
Entry: 16000000
Entry: 17000000
Entry: 18000000
Entry: 19000000
